scale과 C를 섞어서 할 경우 분석을 돌릴 경우 굉장히 오래 걸림

result.rsquared : R square 값  
result.params : coef 값  
result.bse : std 값  
result.aic : AIC  
result.bic : BIC  

result.predict() : 예측 값


In [1]:
import pandas as pd
import numpy as np
import scipy as sp

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import datetime as dt
from patsy import dmatrix

import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes(palette="muted")

%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
from taxi_pakage import *

C:\Users\ledes\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
def pick_features(data, cat_ls, scale_ls, cat = False, scale = False ) :
    """
    data = get features from here(pandas dataframe)
    feautres = location in data set of columns()
    scale = True if you need to scaling 
    cat = True if you need to categorizing
    
    
    """

    cat_features_ls = []
    scale_features_ls = []
    
    #category
    for n in range(len(cat_ls)) : 
        loc = data.columns[cat_ls[n]]
        cat_features_ls.append(loc)
    
    #scale
    for n in range(len(scale_ls)) : 
        loc = data.columns[scale_ls[n]]
        scale_features_ls.append(loc)    
    
    if cat :
        cat_features_ls = list(map(lambda x: "C({})".format(x), cat_features_ls))
        cat_features_ls = " + ".join(cat_features_ls)
    
    if scale :
        scale_features_ls = list(map(lambda x: "scale({})".format(x), scale_features_ls))
        scale_features_ls = " + ".join(scale_features_ls)
        
    return cat_features_ls + " + " +scale_features_ls
    

In [15]:
result_sets = pd.DataFrame(columns = ["R square", "AIC", "BIC", "P_fstatics", "P_omnibus", "P_jb", "Cond_no"])

In [34]:

def storage(result) :
    """
    회귀분석 결과 모음 
    
    전역변수로 아래 데이터 프레임 선언
    result_sets = pd.DataFrame(columns = ["R square", "AIC", "BIC", "P_fstatics", "P_omnibus", "P_jb", "Cond_no"])

    Variables_name.fit() 상태로 삽입
    ex)result = model.fit() 이면
    result를 삽입    
    
    
    reason = 이전 데이터 셋에서 무엇을 변화시켰는지 적을 것.
    """    
    global result_sets 
    result_sets = pd.DataFrame(columns = ["R square", "AIC", "BIC", "P_fstatics", "P_omnibus", "P_jb", "Cond_no"])

    summary = result.summary()
    
    t1 = summary.tables[0]
    t2 = summary.tables[1]
    t3 = summary.tables[2]

    r_squared = float(t1.data[0][3])
    p_fstatics = float(t1.data[3][3])
    
    p_omnibus = float(t3.data[1][1])
    p_jb = float(t3.data[2][3])
    con = float(t3.data[3][3])
    
    put = {
        "R square" : r_squared,
        "AIC" : result.aic,
        "BIC" : result.bic,
        "P_fstatics" : p_fstatics,
        "P_omnibus" : p_omnibus,
        "P_jb" : p_jb,
        "Cond_no" : con,   
    }
    
    result_sets.loc[len(result_sets)] = put



In [4]:
taxi = pd.read_csv('edited_taxi.csv')

In [5]:
feature_n, features = get_features(taxi, start_num = 0, end_num = 14)

In [17]:
#test_index = 0
model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     70.33
Date:                Fri, 09 Mar 2018   Prob (F-statistic):          7.02e-173
Time:                        20:00:52   Log-Likelihood:            -1.4561e+07
No. Observations:             1458644   AIC:                         2.912e+07
Df Residuals:                 1458631   BIC:                         2.912e+07
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
id                 -8.136e-06   3.75e-06     -2.167      0.030   -1.55e-05   -7.78e-07
passenger_count       34.0652      3.301     10.319      0.000      27.595      40.535
pickup_longitude    1936.3637     98.464     19.666      0.000    1743.377    2129.350
pickup_latitude     -746.0211    137.980     -5.407      0.000   -1016.457    -475.585
dropoff_longitude  -1976.4945     95.378    -20.723      0.000   -2163.431   -1789.558
dropoff_latitude     692.6251    129.180      5.362      0.000     439.436     945.814
store_and_fwd_flag   128.2683     58.553      2.191      0.028      13.507     243.029
pick_date              0.6387      0.084      7.576      0.000       0.473       0.804
pick_time             -0.0042      0.001     -8.050      0.000      -0.005      -0.003
drop_time              0.0050      0.001      9.775      0.000       0.004       0.006
holiday              -93.6864     14.713     -6.367      0.000    -122.524     -64.849
holiday_Fri          -26.0587     16.179     -1.611      0.107     -57.769       5.652
weekday               21.5524      4.066      5.301      0.000      13.583      29.521
================================================================================
Omnibus:                  8286956.505   Durbin-Watson:                     2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   2247434660743901.500
Skew:                         343.388   Prob(JB):                           0.00
Kurtosis:                  192299.657   Cond. No.                       9.58e+07
================================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.58e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## F 검정을 사용한 변수 중요도 비교

In [11]:
sm.stats.anova_lm(result, typ = 2)

,sum_sq,df,F,PR(>F)
id,1.300306e+08,1.0,4.786116,2.869021e-02
vendor_id,1.317703e+10,1.0,485.014899,1.803445e-107
passenger_count,1.712296e+08,1.0,6.302550,1.205655e-02
pickup_longitude,2.065528e+08,1.0,7.602715,5.828119e-03
pickup_latitude,4.178426e+08,1.0,15.379784,8.792795e-05
dropoff_longitude,1.239668e+03,1.0,0.000046,9.946104e-01
dropoff_latitude,2.800416e+06,1.0,0.103077,7.481684e-01
store_and_fwd_flag,2.215345e+07,1.0,0.815415,3.665241e-01
dist,3.466845e+11,1.0,12760.628008,0.000000e+00
pick_date,9.821131e+08,1.0,36.149232,1.828137e-09


anova_lm 을 통해 각 변수의 중요도 비교  
dist 의 경우 제외했을 때 오히려 F 검정의 결과값이 낮게 나옴 -> good  
** vendor_id, dist 제외 **

In [18]:
#test_0(index = 0)
#dist, vendor_id 제외
features = 'id + passenger_count + pickup_longitude + pickup_latitude + dropoff_longitude + dropoff_latitude + store_and_fwd_flag + pick_date + pick_time + drop_time + holiday + holiday_Fri + weekday'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
sm.stats.anova_lm(result, tpy = 2)

C:\Users\ledes\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\ledes\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\ledes\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
id,1.0,9.955647e+11,9.955647e+11,36314.526306,0.000000e+00
passenger_count,1.0,1.297423e+11,1.297423e+11,4732.521113,0.000000e+00
pickup_longitude,1.0,2.202007e+11,2.202007e+11,8032.110157,0.000000e+00
pickup_latitude,1.0,1.053239e+08,1.053239e+08,3.841827,4.998922e-02
dropoff_longitude,1.0,1.318264e+10,1.318264e+10,480.853949,1.449457e-106
dropoff_latitude,1.0,7.721414e+08,7.721414e+08,28.164867,1.114248e-07
store_and_fwd_flag,1.0,1.374353e+08,1.374353e+08,5.013131,2.515592e-02
pick_date,1.0,1.786866e+09,1.786866e+09,65.178259,6.847156e-16
pick_time,1.0,2.920643e+08,2.920643e+08,10.653426,1.098697e-03
drop_time,1.0,2.627474e+09,2.627474e+09,95.840557,1.247188e-22


In [19]:
result_sets

,R square,AIC,BIC,P_fstatics,P_omnibus,P_jb,Cond_no
0,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,95800000.0
1,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,95800000.0


### Result_Index 1 - 아웃라이어 제거

In [20]:
#영향도 행렬
influence = result.get_influence()
hat = influence.hat_matrix_diag

StatsModels를 이용한 표준화 잔차 계산

In [21]:
taxi = erase_outlier_np(result, taxi, 13)

In [23]:
#test_1(index = 1)에서 아웃라이어 제거
#test_2(index = 2)
features = 'id + passenger_count + pickup_longitude + pickup_latitude + dropoff_longitude + dropoff_latitude + store_and_fwd_flag + pick_date + pick_time + drop_time + holiday + holiday_Fri + weekday'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     3321.
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        20:38:00   Log-Likelihood:            -1.1488e+07
No. Observations:             1456120   AIC:                         2.298e+07
Df Residuals:                 1456107   BIC:                         2.298e+07
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
id                  7.212e-07   4.64e-07      1.556      0.120   -1.87e-07    1.63e-06
passenger_count        8.3627      0.408     20.506      0.000       7.563       9.162
pickup_longitude    1822.9262     13.407    135.970      0.000    1796.649    1849.203
pickup_latitude     -341.4159     20.556    -16.609      0.000    -381.705    -301.126
dropoff_longitude  -1884.8944     12.603   -149.556      0.000   -1909.596   -1860.193
dropoff_latitude     246.6601     17.337     14.227      0.000     212.680     280.640
store_and_fwd_flag   162.3967      7.303     22.237      0.000     148.083     176.711
pick_date              0.7136      0.010     68.552      0.000       0.693       0.734
pick_time             -0.0013   6.44e-05    -19.837      0.000      -0.001      -0.001
drop_time              0.0019   6.36e-05     29.488      0.000       0.002       0.002
holiday             -116.4119      1.817    -64.064      0.000    -119.973    -112.850
holiday_Fri          -30.9462      1.998    -15.490      0.000     -34.862     -27.030
weekday               16.9306      0.502     33.723      0.000      15.947      17.915
==============================================================================
Omnibus:                   713383.127   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5932575.757
Skew:                           2.205   Prob(JB):                         0.00
Kurtosis:                      11.851   Cond. No.                     1.05e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.05e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
sm.stats.anova_lm(result, tpy = 2)

C:\Users\ledes\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\ledes\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\ledes\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
id,1.0,7.634216e+11,7.634216e+11,1.829410e+06,0.000000e+00
passenger_count,1.0,7.788579e+10,7.788579e+10,1.866400e+05,0.000000e+00
pickup_longitude,1.0,1.754827e+11,1.754827e+11,4.205143e+05,0.000000e+00
pickup_latitude,1.0,5.128047e+08,5.128047e+08,1.228849e+03,4.250156e-269
dropoff_longitude,1.0,1.004276e+10,1.004276e+10,2.406575e+04,0.000000e+00
dropoff_latitude,1.0,8.652399e+07,8.652399e+07,2.073400e+02,5.265754e-47
store_and_fwd_flag,1.0,2.121458e+08,2.121458e+08,5.083713e+02,1.499392e-112
pick_date,1.0,2.303202e+09,2.303202e+09,5.519229e+03,0.000000e+00
pick_time,1.0,3.153731e+08,3.153731e+08,7.557378e+02,2.503176e-166
drop_time,1.0,3.880713e+08,3.880713e+08,9.299466e+02,3.519705e-204


Result_index2 - 변수 중 id, passenger_count, pickup_longitude 제외

In [25]:
#test_3(index = 3)
features = 'pickup_latitude + dropoff_longitude + dropoff_latitude + store_and_fwd_flag + pick_date + pick_time + drop_time + holiday + holiday_Fri + weekday'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.625
Method:                 Least Squares   F-statistic:                 2.426e+05
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        20:40:26   Log-Likelihood:            -1.1498e+07
No. Observations:             1456120   AIC:                         2.300e+07
Df Residuals:                 1456110   BIC:                         2.300e+07
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
pickup_latitude    -1218.7346     19.646    -62.034      0.000   -1257.240   -1180.229
dropoff_longitude   -621.3868      8.564    -72.556      0.000    -638.172    -604.601
dropoff_latitude     108.7537     17.419      6.243      0.000      74.612     142.895
store_and_fwd_flag   165.7335      7.348     22.553      0.000     151.331     180.136
pick_date              0.7127      0.010     68.026      0.000       0.692       0.733
pick_time             -0.0013   6.48e-05    -20.240      0.000      -0.001      -0.001
drop_time              0.0021    6.4e-05     32.426      0.000       0.002       0.002
holiday             -116.9899      1.829    -63.975      0.000    -120.574    -113.406
holiday_Fri          -30.5172      2.011    -15.177      0.000     -34.458     -26.576
weekday               15.7240      0.505     31.124      0.000      14.734      16.714
==============================================================================
Omnibus:                   715415.342   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5869678.671
Skew:                           2.218   Prob(JB):                         0.00
Kurtosis:                      11.779   Cond. No.                     3.51e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.51e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- R square 0.6으로 대폭 상승
- 조건수를 낮춰야 할까, 아웃라이어를 제거해야할까, 변수를 조절해야 할까
- 이 상황에서 정답이 있을까?
- 아웃라이어 제거 이후 같은 변수로 한 번 더 분석

In [29]:
#문수님 패키지 사용
taxi = erase_outlier_np(result = result, data = taxi, total_feature = 10 + 1)

In [31]:
#test_4(index = 4)
features = 'pickup_latitude + dropoff_longitude + dropoff_latitude + store_and_fwd_flag + pick_date + pick_time + drop_time + holiday + holiday_Fri + weekday'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.724
Method:                 Least Squares   F-statistic:                 3.597e+05
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        20:54:39   Log-Likelihood:            -1.0361e+07
No. Observations:             1374072   AIC:                         2.072e+07
Df Residuals:                 1374062   BIC:                         2.072e+07
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
pickup_latitude      123.5322     18.584      6.647      0.000      87.109     159.956
dropoff_longitude   -215.6928      7.064    -30.533      0.000    -229.539    -201.847
dropoff_latitude    -498.8254     15.571    -32.036      0.000    -529.344    -468.307
store_and_fwd_flag   234.1244      9.576     24.450      0.000     215.357     252.892
pick_date              0.4275      0.008     56.501      0.000       0.413       0.442
pick_time             -0.0057   6.53e-05    -86.925      0.000      -0.006      -0.006
drop_time              0.0063   6.49e-05     96.722      0.000       0.006       0.006
holiday              -73.8296      1.327    -55.622      0.000     -76.431     -71.228
holiday_Fri          -62.2943      1.461    -42.639      0.000     -65.158     -59.431
weekday               17.1739      0.368     46.727      0.000      16.454      17.894
==============================================================================
Omnibus:                   179881.375   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           261712.581
Skew:                           0.995   Prob(JB):                         0.00
Kurtosis:                       3.784   Cond. No.                     4.61e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.61e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [32]:
result_sets

,R square,AIC,BIC,P_fstatics,P_omnibus,P_jb,Cond_no
0,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,95800000.0
1,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,95800000.0
2,0.027,2.297679e+07,2.297694e+07,0.000000e+00,0.0,0.0,105000000.0
3,0.625,2.299558e+07,2.299571e+07,0.000000e+00,0.0,0.0,3510000.0
4,0.724,2.072157e+07,2.072169e+07,0.000000e+00,0.0,0.0,4610000.0


r square의 증가 - 계속되는 outlier 제거로 인한 듯.  
감소하던 조건수가 증가 - scaling으로 감소 시도  

In [35]:
#변수 동일, 실수데이터에만 scaling
#test_5(index = 5)
features = 'scale(pickup_latitude) + scale(dropoff_longitude) + scale(dropoff_latitude) + store_and_fwd_flag + pick_date + pick_time + drop_time + holiday + holiday_Fri + weekday'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.688
Method:                 Least Squares   F-statistic:                 3.026e+05
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:07:25   Log-Likelihood:            -1.0445e+07
No. Observations:             1374072   AIC:                         2.089e+07
Df Residuals:                 1374062   BIC:                         2.089e+07
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
scale(pickup_latitude)     -16.4025      0.504    -32.557      0.000     -17.390     -15.415
scale(dropoff_longitude)    75.6083      0.434    174.158      0.000      74.757      76.459
scale(dropoff_latitude)    -60.5369      0.524   -115.513      0.000     -61.564     -59.510
store_and_fwd_flag         276.7599     10.178     27.193      0.000     256.812     296.708
pick_date                    2.1793      0.007    311.357      0.000       2.166       2.193
pick_time                   -0.0029   6.92e-05    -41.928      0.000      -0.003      -0.003
drop_time                    0.0091   6.87e-05    131.874      0.000       0.009       0.009
holiday                   -134.7723      1.405    -95.921      0.000    -137.526    -132.019
holiday_Fri               -131.8303      1.545    -85.338      0.000    -134.858    -128.803
weekday                     89.0597      0.355    250.695      0.000      88.363      89.756
==============================================================================
Omnibus:                   113001.106   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           147922.259
Skew:                           0.722   Prob(JB):                         0.00
Kurtosis:                       3.706   Cond. No.                     1.94e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.94e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [36]:
result_sets

,R square,AIC,BIC,P_fstatics,P_omnibus,P_jb,Cond_no
0,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,95800000.0
1,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,95800000.0
2,0.027,2.297679e+07,2.297694e+07,0.000000e+00,0.0,0.0,105000000.0
3,0.625,2.299558e+07,2.299571e+07,0.000000e+00,0.0,0.0,3510000.0
4,0.724,2.072157e+07,2.072169e+07,0.000000e+00,0.0,0.0,4610000.0
5,0.688,2.088935e+07,2.088947e+07,0.000000e+00,0.0,0.0,1940000.0


- 조건수, R square 모두 감소, aic 및 bic 여전히 높음.
- 변수 제거

In [37]:
sm.stats.anova_lm(result, tpy = 2)

C:\Users\ledes\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\ledes\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\ledes\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(pickup_latitude),1.0,2.367456e+09,2.367456e+09,1.010165e+04,0.0
scale(dropoff_longitude),1.0,3.940270e+09,3.940270e+09,1.681266e+04,0.0
scale(dropoff_latitude),1.0,2.846329e+09,2.846329e+09,1.214494e+04,0.0
store_and_fwd_flag,1.0,2.072386e+09,2.072386e+09,8.842619e+03,0.0
pick_date,1.0,5.791863e+11,5.791863e+11,2.471318e+06,0.0
pick_time,1.0,9.578933e+10,9.578933e+10,4.087215e+05,0.0
drop_time,1.0,4.750647e+09,4.750647e+09,2.027044e+04,0.0
holiday,1.0,1.557934e+09,1.557934e+09,6.647515e+03,0.0
holiday_Fri,1.0,1.879722e+09,1.879722e+09,8.020547e+03,0.0
weekday,1.0,1.472926e+10,1.472926e+10,6.284796e+04,0.0


- 여지껏 변수를 선별할 때는 F 검정을 활용했는데 난감
- test4에서 카테고리함수를 씌운후 계산해보면 어떨까?

> 예상 : 변수 증가하니 R.adj, aic, bic 감소 있을 것

In [40]:
result_sets

,R square,AIC,BIC,P_fstatics,P_omnibus,P_jb,Cond_no
0,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,95800000.0
1,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,95800000.0
2,0.027,2.297679e+07,2.297694e+07,0.000000e+00,0.0,0.0,105000000.0
3,0.625,2.299558e+07,2.299571e+07,0.000000e+00,0.0,0.0,3510000.0
4,0.724,2.072157e+07,2.072169e+07,0.000000e+00,0.0,0.0,4610000.0
5,0.688,2.088935e+07,2.088947e+07,0.000000e+00,0.0,0.0,1940000.0


In [41]:
#실험 과정 중 실수 발견 - 카테고리화를 했음에도 불구하고, 
#test_5 : test_4에서 스케일링 추가, test_6 : test_4에서 카테고리화 추가(동시에는 하지 않음)
#test_6(index = 6)
features = 'pickup_latitude + dropoff_longitude + dropoff_latitude + C(store_and_fwd_flag) + C(pick_date) + pick_time + drop_time + C(holiday) + C(holiday_Fri) + C(weekday)'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     416.3
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:33:30   Log-Likelihood:            -1.0335e+07
No. Observations:             1374072   AIC:                         2.067e+07
Df Residuals:                 1373878   BIC:                         2.067e+07
Df Model:                         193                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(store_and_fwd_flag)[0]  2.775e+05   1311.198    211.659      0.000    2.75e+05     2.8e+05
C(store_and_fwd_flag)[1]  2.778e+05   1311.188    211.832      0.000    2.75e+05     2.8e+05
C(pick_date)[T.1]         -406.0124    332.361     -1.222      0.222   -1057.428     245.404
C(pick_date)[T.2]         -823.3417    387.699     -2.124      0.034   -1583.219     -63.465
C(pick_date)[T.3]         9732.0868    249.023     39.081      0.000    9244.010    1.02e+04
C(pick_date)[T.4]         9509.8024    211.781     44.904      0.000    9094.719    9924.886
C(pick_date)[T.5]         1368.2126    412.539      3.317      0.001     559.650    2176.775
C(pick_date)[T.6]          879.9907    332.358      2.648      0.008     228.580    1531.401
C(pick_date)[T.7]          108.2283    267.542      0.405      0.686    -416.145     632.601
C(pick_date)[T.8]         -380.6752    332.344     -1.145      0.252   -1032.058     270.707
C(pick_date)[T.9]         -820.8925    387.692     -2.117      0.034   -1580.756     -61.029
C(pick_date)[T.10]        9772.7278    249.021     39.245      0.000    9284.656    1.03e+04
C(pick_date)[T.11]        9544.1460    211.776     45.067      0.000    9129.071    9959.221
C(pick_date)[T.12]        1383.9353    412.511      3.355      0.001     575.428    2192.442
C(pick_date)[T.13]         901.8083    332.355      2.713      0.007     250.403    1553.213
C(pick_date)[T.14]         124.2441    267.579      0.464      0.642    -400.202     648.691
C(pick_date)[T.15]        -377.4402    332.348     -1.136      0.256   -1028.831     273.951
C(pick_date)[T.16]        -800.4665    387.690     -2.065      0.039   -1560.326     -40.607
C(pick_date)[T.17]        1.007e+04    352.838     28.545      0.000    9380.196    1.08e+04
C(pick_date)[T.18]        9582.0615    211.777     45.246      0.000    9166.985    9997.138
C(pick_date)[T.19]        1419.7426    412.536      3.441      0.001     611.186    2228.300
C(pick_date)[T.20]         927.8071    332.355      2.792      0.005     276.402    1579.212
C(pick_date)[T.21]         112.9345    267.578      0.422      0.673    -411.509     637.378
C(pick_date)[T.22]        -320.0896    332.507     -0.963      0.336    -971.791     331.612
C(pick_date)[T.23]        -780.1344    387.735     -2.012      0.044   -1540.082     -20.187
C(pick_date)[T.24]        9916.9735    249.028     39.823      0.000    9428.887    1.04e+04
C(pick_date)[T.25]        9730.5787    211.784     45.946      0.000    9315.490    1.01e+04
C(pick_date)[T.26]        1515.4642    412.539      3.674      0.000     706.902    2324.026
C(pick_date)[T.27]         973.3464    332.357      2.929      0.003     321.939    1624.754
C(pick_date)[T.28]         178.1876    267.578      0.666      0.505    -346.257     702.632
C(pick_date)[T

In [42]:
result_sets

,R square,AIC,BIC,P_fstatics,P_omnibus,P_jb,Cond_no
0,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,9.580000e+07
1,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,9.580000e+07
2,0.027,2.297679e+07,2.297694e+07,0.000000e+00,0.0,0.0,1.050000e+08
3,0.625,2.299558e+07,2.299571e+07,0.000000e+00,0.0,0.0,3.510000e+06
4,0.724,2.072157e+07,2.072169e+07,0.000000e+00,0.0,0.0,4.610000e+06
5,0.688,2.088935e+07,2.088947e+07,0.000000e+00,0.0,0.0,1.940000e+06
6,0.055,2.066955e+07,2.067191e+07,0.000000e+00,0.0,0.0,7.450000e+17


- 변수가 많아 끔찍하게 오래 걸림
- R Square, 조건수 모두 난리 남. -> 섣부른 카테고리화는 큰 화를 부른다.  
- test_4로 회귀하여 아웃라이어 제거할 것.

In [51]:
#test_4의 result 값을 얻기 위해 한 번 더 실행
#test_4(index = 4)
features = 'pickup_latitude + dropoff_longitude + dropoff_latitude + store_and_fwd_flag + pick_date + pick_time + drop_time + holiday + holiday_Fri + weekday'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.724
Model:                            OLS   Adj. R-squared:                  0.724
Method:                 Least Squares   F-statistic:                 3.597e+05
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:55:27   Log-Likelihood:            -1.0361e+07
No. Observations:             1374072   AIC:                         2.072e+07
Df Residuals:                 1374062   BIC:                         2.072e+07
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
pickup_latitude      123.5322     18.584      6.647      0.000      87.109     159.956
dropoff_longitude   -215.6928      7.064    -30.533      0.000    -229.539    -201.847
dropoff_latitude    -498.8254     15.571    -32.036      0.000    -529.344    -468.307
store_and_fwd_flag   234.1244      9.576     24.450      0.000     215.357     252.892
pick_date              0.4275      0.008     56.501      0.000       0.413       0.442
pick_time             -0.0057   6.53e-05    -86.925      0.000      -0.006      -0.006
drop_time              0.0063   6.49e-05     96.722      0.000       0.006       0.006
holiday              -73.8296      1.327    -55.622      0.000     -76.431     -71.228
holiday_Fri          -62.2943      1.461    -42.639      0.000     -65.158     -59.431
weekday               17.1739      0.368     46.727      0.000      16.454      17.894
==============================================================================
Omnibus:                   179881.375   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           261712.581
Skew:                           0.995   Prob(JB):                         0.00
Kurtosis:                       3.784   Cond. No.                     4.61e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.61e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [53]:
taxi = erase_outlier_np(result = result, data = taxi, total_feature = 10 + 1)

In [55]:
#test_4에서 아웃라이어 제거 후 동일 변수에서 실행
#test_7(index = 7)
features = 'pickup_latitude + dropoff_longitude + dropoff_latitude + store_and_fwd_flag + pick_date + pick_time + drop_time + holiday + holiday_Fri + weekday'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.766
Method:                 Least Squares   F-statistic:                 4.247e+05
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:57:47   Log-Likelihood:            -9.5667e+06
No. Observations:             1300483   AIC:                         1.913e+07
Df Residuals:                 1300473   BIC:                         1.913e+07
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
pickup_latitude      406.3777     17.749     22.896      0.000     371.591     441.165
dropoff_longitude   -239.6334      6.280    -38.159      0.000    -251.942    -227.325
dropoff_latitude    -826.9120     15.107    -54.738      0.000    -856.521    -797.303
store_and_fwd_flag   274.9559     12.728     21.603      0.000     250.010     299.902
pick_date              0.3103      0.006     47.904      0.000       0.298       0.323
pick_time             -0.0403      0.000   -243.607      0.000      -0.041      -0.040
drop_time              0.0410      0.000    248.029      0.000       0.041       0.041
holiday              -50.6061      1.142    -44.308      0.000     -52.845     -48.368
holiday_Fri          -66.8936      1.258    -53.190      0.000     -69.359     -64.429
weekday               15.3943      0.316     48.706      0.000      14.775      16.014
==============================================================================
Omnibus:                   227741.385   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           541455.052
Skew:                           0.996   Prob(JB):                         0.00
Kurtosis:                       5.455   Cond. No.                     5.24e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.24e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [56]:
result_sets

,R square,AIC,BIC,P_fstatics,P_omnibus,P_jb,Cond_no
0,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,9.580000e+07
1,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,9.580000e+07
2,0.027,2.297679e+07,2.297694e+07,0.000000e+00,0.0,0.0,1.050000e+08
3,0.625,2.299558e+07,2.299571e+07,0.000000e+00,0.0,0.0,3.510000e+06
4,0.724,2.072157e+07,2.072169e+07,0.000000e+00,0.0,0.0,4.610000e+06
5,0.688,2.088935e+07,2.088947e+07,0.000000e+00,0.0,0.0,1.940000e+06
6,0.055,2.066955e+07,2.067191e+07,0.000000e+00,0.0,0.0,7.450000e+17
7,0.766,1.913346e+07,1.913358e+07,0.000000e+00,0.0,0.0,5.240000e+06


4와 7의 비교 
- R square의 증가
- 조건수도 증가 : 아웃라이어만 제거했을 뿐인데 조건수가 증가했다?

변수 중 pick_date, pick_time, drop_time은 큰 영향을 끼치지 않는 것을 발견 -> 제외

In [57]:
#변수 중 pick_date, pick_time, drop_time 삭제
#test_8(index = 8)
features = 'pickup_latitude + dropoff_longitude + dropoff_latitude + store_and_fwd_flag + holiday + holiday_Fri + weekday'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.754
Method:                 Least Squares   F-statistic:                 5.688e+05
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        22:11:25   Log-Likelihood:            -9.5987e+06
No. Observations:             1300483   AIC:                         1.920e+07
Df Residuals:                 1300476   BIC:                         1.920e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
pickup_latitude      433.3827     18.190     23.826      0.000     397.732     469.034
dropoff_longitude   -245.6842      6.436    -38.175      0.000    -258.298    -233.070
dropoff_latitude    -862.6961     15.481    -55.725      0.000    -893.039    -832.353
store_and_fwd_flag   288.5235     13.044     22.119      0.000     262.958     314.089
holiday              -56.8861      1.169    -48.682      0.000     -59.176     -54.596
holiday_Fri          -70.9115      1.288    -55.045      0.000     -73.436     -68.387
weekday               16.0550      0.323     49.631      0.000      15.421      16.689
==============================================================================
Omnibus:                    95872.426   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           118992.449
Skew:                           0.741   Prob(JB):                         0.00
Kurtosis:                       3.045   Cond. No.                     6.23e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.23e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [58]:
result_sets

,R square,AIC,BIC,P_fstatics,P_omnibus,P_jb,Cond_no
0,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,9.580000e+07
1,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,9.580000e+07
2,0.027,2.297679e+07,2.297694e+07,0.000000e+00,0.0,0.0,1.050000e+08
3,0.625,2.299558e+07,2.299571e+07,0.000000e+00,0.0,0.0,3.510000e+06
4,0.724,2.072157e+07,2.072169e+07,0.000000e+00,0.0,0.0,4.610000e+06
5,0.688,2.088935e+07,2.088947e+07,0.000000e+00,0.0,0.0,1.940000e+06
6,0.055,2.066955e+07,2.067191e+07,0.000000e+00,0.0,0.0,7.450000e+17
7,0.766,1.913346e+07,1.913358e+07,0.000000e+00,0.0,0.0,5.240000e+06
8,0.754,1.919734e+07,1.919742e+07,0.000000e+00,0.0,0.0,6.230000e+03


- R sqaure 소량 감소, 조건수 증가
- 스케일링 시도

In [59]:
#test_9(index = 9)
features = 'scale(pickup_latitude) + scale(dropoff_longitude) + scale(dropoff_latitude) + store_and_fwd_flag + holiday + holiday_Fri + weekday'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.574
Model:                            OLS   Adj. R-squared:                  0.574
Method:                 Least Squares   F-statistic:                 2.503e+05
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        22:15:26   Log-Likelihood:            -9.9551e+06
No. Observations:             1300483   AIC:                         1.991e+07
Df Residuals:                 1300476   BIC:                         1.991e+07
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
scale(pickup_latitude)       8.8089      0.578     15.228      0.000       7.675       9.943
scale(dropoff_longitude)    37.2138      0.476     78.167      0.000      36.281      38.147
scale(dropoff_latitude)    -48.5080      0.606    -80.029      0.000     -49.696     -47.320
store_and_fwd_flag         459.8987     17.156     26.807      0.000     426.273     493.524
holiday                   -343.0597      1.490   -230.302      0.000    -345.979    -340.140
holiday_Fri               -341.4999      1.655   -206.319      0.000    -344.744    -338.256
weekday                    253.5477      0.283    896.358      0.000     252.993     254.102
==============================================================================
Omnibus:                    61572.799   Durbin-Watson:                   1.795
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            70829.675
Skew:                           0.572   Prob(JB):                         0.00
Kurtosis:                       3.001   Cond. No.                         142.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
result_sets

,R square,AIC,BIC,P_fstatics,P_omnibus,P_jb,Cond_no
0,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,9.580000e+07
1,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,9.580000e+07
2,0.027,2.297679e+07,2.297694e+07,0.000000e+00,0.0,0.0,1.050000e+08
3,0.625,2.299558e+07,2.299571e+07,0.000000e+00,0.0,0.0,3.510000e+06
4,0.724,2.072157e+07,2.072169e+07,0.000000e+00,0.0,0.0,4.610000e+06
5,0.688,2.088935e+07,2.088947e+07,0.000000e+00,0.0,0.0,1.940000e+06
6,0.055,2.066955e+07,2.067191e+07,0.000000e+00,0.0,0.0,7.450000e+17
7,0.766,1.913346e+07,1.913358e+07,0.000000e+00,0.0,0.0,5.240000e+06
8,0.754,1.919734e+07,1.919742e+07,0.000000e+00,0.0,0.0,6.230000e+03
9,0.574,1.991030e+07,1.991038e+07,0.000000e+00,0.0,0.0,1.420000e+02


- 조건수 대폭 감소, R square도 대폭 감소...
- 위도, 경도를 스케일링 하는 것은 좋지 않은 듯

In [61]:
#성인님 VIF 참고, holiday 제거
#test_10(index = 10)
features = 'pickup_latitude + dropoff_longitude + dropoff_latitude + store_and_fwd_flag + holiday_Fri + weekday'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.753
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                 6.620e+05
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        22:28:41   Log-Likelihood:            -9.5998e+06
No. Observations:             1300483   AIC:                         1.920e+07
Df Residuals:                 1300477   BIC:                         1.920e+07
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
pickup_latitude      444.9665     18.205     24.443      0.000     409.286     480.647
dropoff_longitude   -240.9172      6.441    -37.404      0.000    -253.541    -228.293
dropoff_latitude    -865.4226     15.495    -55.851      0.000    -895.793    -835.052
store_and_fwd_flag   288.6141     13.056     22.106      0.000     263.025     314.203
holiday_Fri          -93.5099      1.203    -77.743      0.000     -95.867     -91.152
weekday               10.8389      0.306     35.478      0.000      10.240      11.438
==============================================================================
Omnibus:                    96090.112   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           119330.790
Skew:                           0.742   Prob(JB):                         0.00
Kurtosis:                       3.039   Cond. No.                     6.23e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.23e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [62]:
result_sets

,R square,AIC,BIC,P_fstatics,P_omnibus,P_jb,Cond_no
0,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,9.580000e+07
1,0.001,2.912108e+07,2.912124e+07,7.020000e-173,0.0,0.0,9.580000e+07
2,0.027,2.297679e+07,2.297694e+07,0.000000e+00,0.0,0.0,1.050000e+08
3,0.625,2.299558e+07,2.299571e+07,0.000000e+00,0.0,0.0,3.510000e+06
4,0.724,2.072157e+07,2.072169e+07,0.000000e+00,0.0,0.0,4.610000e+06
5,0.688,2.088935e+07,2.088947e+07,0.000000e+00,0.0,0.0,1.940000e+06
6,0.055,2.066955e+07,2.067191e+07,0.000000e+00,0.0,0.0,7.450000e+17
7,0.766,1.913346e+07,1.913358e+07,0.000000e+00,0.0,0.0,5.240000e+06
8,0.754,1.919734e+07,1.919742e+07,0.000000e+00,0.0,0.0,6.230000e+03
9,0.574,1.991030e+07,1.991038e+07,0.000000e+00,0.0,0.0,1.420000e+02


- r square 증가, 조건수 증가
- store_and_fwd_flag, holiday_Fri, weekday 카테고리화

In [63]:
#store_and_fwd_flag, holiday_Fri, weekday 카테고리화 제거
#test_11(index = 11)
features = 'pickup_latitude + dropoff_longitude + dropoff_latitude + C(store_and_fwd_flag) + C(holiday_Fri) + C(weekday)'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     3369.
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        22:37:56   Log-Likelihood:            -9.5879e+06
No. Observations:             1300483   AIC:                         1.918e+07
Df Residuals:                 1300471   BIC:                         1.918e+07
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(store_and_fwd_flag)[0]  1.818e+05   1313.795    138.416      0.000    1.79e+05    1.84e+05
C(store_and_fwd_flag)[1]  1.821e+05   1313.810    138.629      0.000     1.8e+05    1.85e+05
C(holiday_Fri)[T.1]       -147.5457      3.171    -46.532      0.000    -153.760    -141.331
C(weekday)[T.1]             51.5537      1.342     38.426      0.000      48.924      54.183
C(weekday)[T.2]             65.3649      1.332     49.063      0.000      62.754      67.976
C(weekday)[T.3]             56.9804      1.327     42.942      0.000      54.380      59.581
C(weekday)[T.4]            158.1011      3.135     50.425      0.000     151.956     164.246
C(weekday)[T.5]            141.7618      3.132     45.268      0.000     135.624     147.900
C(weekday)[T.6]            109.0415      3.147     34.645      0.000     102.873     115.210
pickup_latitude             13.6889     18.295      0.748      0.454     -22.170      49.547
dropoff_longitude         1452.1577     13.804    105.198      0.000    1425.102    1479.213
dropoff_latitude         -1823.7211     16.865   -108.134      0.000   -1856.777   -1790.666
==============================================================================
Omnibus:                    90910.169   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           111367.418
Skew:                           0.714   Prob(JB):                         0.00
Kurtosis:                       3.131   Cond. No.                     5.16e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.16e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- 쓰렉...store_and_fwd_flag만 카테고리 해제

In [64]:
#store_and_fwd_flag만 카테고리 해제
#test_12(index = 12)
features = 'pickup_latitude + dropoff_longitude + dropoff_latitude + store_and_fwd_flag + C(holiday_Fri) + C(weekday)'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     3369.
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        22:39:49   Log-Likelihood:            -9.5879e+06
No. Observations:             1300483   AIC:                         1.918e+07
Df Residuals:                 1300471   BIC:                         1.918e+07
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(holiday_Fri)[0]   1.818e+05   1313.795    138.416      0.000    1.79e+05    1.84e+05
C(holiday_Fri)[1]   1.817e+05   1313.817    138.301      0.000    1.79e+05    1.84e+05
C(weekday)[T.1]       51.5537      1.342     38.426      0.000      48.924      54.183
C(weekday)[T.2]       65.3649      1.332     49.063      0.000      62.754      67.976
C(weekday)[T.3]       56.9804      1.327     42.942      0.000      54.380      59.581
C(weekday)[T.4]      158.1011      3.135     50.425      0.000     151.956     164.246
C(weekday)[T.5]      141.7618      3.132     45.268      0.000     135.624     147.900
C(weekday)[T.6]      109.0415      3.147     34.645      0.000     102.873     115.210
pickup_latitude       13.6889     18.295      0.748      0.454     -22.170      49.547
dropoff_longitude   1452.1577     13.804    105.198      0.000    1425.102    1479.213
dropoff_latitude   -1823.7211     16.865   -108.134      0.000   -1856.777   -1790.666
store_and_fwd_flag   281.7783     12.937     21.781      0.000     256.422     307.134
==============================================================================
Omnibus:                    90910.169   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           111367.418
Skew:                           0.714   Prob(JB):                         0.00
Kurtosis:                       3.131   Cond. No.                     5.16e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.16e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- holiday_Fri에 너무 많은 가중치 할당 - 카테고리 해제

In [65]:
#holiday_Fri 카테고리 해제
#test_13(index = 13)
features = 'pickup_latitude + dropoff_longitude + dropoff_latitude + store_and_fwd_flag + holiday_Fri + C(weekday)'

model =  sm.OLS.from_formula("trip_duration ~ {} + 0".format(features), data = taxi)
result = model.fit()
storage(result)
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          trip_duration   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     3369.
Date:                Fri, 09 Mar 2018   Prob (F-statistic):               0.00
Time:                        22:40:57   Log-Likelihood:            -9.5879e+06
No. Observations:             1300483   AIC:                         1.918e+07
Df Residuals:                 1300471   BIC:                         1.918e+07
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(weekday)[0]       1.818e+05   1313.795    138.416      0.000    1.79e+05    1.84e+05
C(weekday)[1]       1.819e+05   1313.801    138.454      0.000    1.79e+05    1.84e+05
C(weekday)[2]       1.819e+05   1313.802    138.465      0.000    1.79e+05    1.84e+05
C(weekday)[3]       1.819e+05   1313.800    138.458      0.000    1.79e+05    1.84e+05
C(weekday)[4]        1.82e+05   1313.785    138.537      0.000    1.79e+05    1.85e+05
C(weekday)[5]        1.82e+05   1313.775    138.526      0.000    1.79e+05    1.85e+05
C(weekday)[6]        1.82e+05   1313.757    138.503      0.000    1.79e+05    1.85e+05
pickup_latitude       13.6889     18.295      0.748      0.454     -22.170      49.547
dropoff_longitude   1452.1577     13.804    105.198      0.000    1425.102    1479.213
dropoff_latitude   -1823.7211     16.865   -108.134      0.000   -1856.777   -1790.666
store_and_fwd_flag   281.7783     12.937     21.781      0.000     256.422     307.134
holiday_Fri         -147.5457      3.171    -46.532      0.000    -153.760    -141.331
==============================================================================
Omnibus:                    90910.169   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           111367.418
Skew:                           0.714   Prob(JB):                         0.00
Kurtosis:                       3.131   Cond. No.                     9.65e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.65e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

음 망했구만 하하하하하하